# Freely-Propagating Adiabatic Flame

Code is ported (almost) verbatim from cantera example file `adiabatic_flame.py` (distributed with cantera):
```
"""
A freely-propagating, premixed hydrogen flat flame with multicomponent
transport properties.
"""
```

In [1]:
import cantera as ct
import numpy as np

### Specify Parameters and Objects

In [4]:
# Simulation parameters
p = ct.one_atm  # pressure [Pa]
Tin = 300.0  # unburned gas temperature [K]
reactants = 'H2:1.1, O2:1, AR:5'  # premixed gas composition
width = 0.03  # m
loglevel = 1  # amount of diagnostic output (0 to 8)

# IdealGasMix object used to compute mixture properties, set to the state of the
# upstream fuel-air mixture
gas = ct.Solution('h2o2.xml')
gas.TPX = Tin, p, reactants

# Set up flame object
f = ct.FreeFlame(gas, width=width)
f.set_refine_criteria(ratio=3, slope=0.06, curve=0.12)
print('state after initialization:')
f.show_solution()

state after initialization:


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> reactants <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    Mass Flux:        1.339 kg/m^2/s 
    Temperature:        300 K 
    Mass Fractions: 
                      H2    0.009478 
                      O2      0.1368 
                      AR      0.8537 



>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> flame <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    Pressure:   1.013e+05 Pa

-------------------------------------------------------------------------------
          z           u           V           T      lambda          H2 
-------------------------------------------------------------------------------
          0           1           0         300           0    0.009478 
      0.006           1           0         300           0    0.009478 
      0.009           1           0         300           0    0.009478 
     0.0105       2.206           0       698.2           0     0.00711 
      0.012       3.411           0        1096       

### Solve with Mixture-Averaged Transport Model

In [5]:
# Solve with mixture-averaged transport model
f.transport_model = 'Mix'
f.solve(loglevel=loglevel, auto=True)

# Solve with the energy equation enabled
f.save('h2_adiabatic.xml', 'mix', 'solution with mixture-averaged transport')
f.show_solution()
print('mixture-averaged flamespeed = {0:7f} m/s'.format(f.u[0]))


************ Solving on 9 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.898e-05      6.481
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point grid(s).

..............................................................................
grid refinement disabled.

******************** Solving with grid refinement enabled ********************

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 0 1 2 3 4 5 6 
    to resolve AR H H2 H2

### Multi-Component Transport

Warning: when simulating hydrocarbon flames, this option is considerably slower than mixture-averaged!

In [6]:
# Solve with multi-component transport properties
f.transport_model = 'Multi'
f.solve(loglevel) # don't use 'auto' on subsequent solves
f.show_solution()
print('multicomponent flamespeed = {0:7f} m/s'.format(f.u[0]))
f.save('h2_adiabatic.xml','multi', 'solution with multicomponent transport')

# write the velocity, temperature, density, and mole fractions to a CSV file
f.write_csv('h2_adiabatic.csv', quiet=False)


..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [135] point grid(s).

..............................................................................
no new points needed in flame


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> reactants <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    Mass Flux:        0.973 kg/m^2/s 
    Temperature:        300 K 
    Mass Fractions: 
                      H2    0.009478 
                      O2      0.1368 
                      AR      0.8537 



>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> flame <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    Pressure:   1.013e+05 Pa

-------------------------------------------------------------------------------
          z           u           V           T      lambda          H2 
-------------------------------------------------------------------------------
          0      0.7269           0         300           0    0.009478 
      0.